In [1]:
import pandas as pd
import numpy as np
import altair as alt
import re

# Fundamentals of Data Visualization Final Project

DataSet is from https://www.vizforsocialgood.com/join-a-project/2022/9/15/undp-accelerator-labs-network  that wants wants to increase acces to affordable and clean energy around the world. The dataset contains 359 grassroots energy solutions from across different regions, demographics and energy sources. It also contains a page per region specifiyng which country belong to which region. The regions are Africa, Asia Pacific, Arab States Europe and the Commonwealth of Independent States and Latin America. 

In [2]:
#Load the main data as well as the regions data.

Ed = pd.read_excel('Viz4SocialGood_submissionV1.xlsx', sheet_name = 'Energy_sol_data')
RBA = pd.read_excel('Viz4SocialGood_submissionV1.xlsx', sheet_name = 'RB_Africa')
RBAP = pd.read_excel('Viz4SocialGood_submissionV1.xlsx', sheet_name = 'RB_Asia_Pacific')
RBAS = pd.read_excel('Viz4SocialGood_submissionV1.xlsx', sheet_name = 'RB_Arab States')
RBEC = pd.read_excel('Viz4SocialGood_submissionV1.xlsx', sheet_name = 'RB_Europe_CIS')
RBLAC = pd.read_excel('Viz4SocialGood_submissionV1.xlsx', sheet_name = 'RB_Latin America')


In [3]:

RBA['Region'] = re.sub('Regional Bureau for ','',RBA.columns.values[0]).rsplit(' ', 1)[0]
RBA.rename(columns = {RBA.columns[0]:"Regional_Bureau" }, inplace = True)

RBAP['Region'] = re.sub('Regional Bureau for ','',RBAP.columns.values[0]).rsplit(' ', 1)[0]
RBAP.rename(columns = {RBAP.columns[0]:"Regional_Bureau" }, inplace = True)

RBAS['Region'] = re.sub('Regional Bureau for ','',RBAS.columns.values[0]).rsplit(' ', 1)[0]
RBAS.rename(columns = {RBAS.columns[0]:"Regional_Bureau" }, inplace = True)

RBEC['Region'] = re.sub('Regional Bureau for ','',RBEC.columns.values[0]).rsplit(' ', 1)[0]
RBEC.rename(columns = {RBEC.columns[0]:"Regional_Bureau" }, inplace = True)

RBLAC['Region'] = re.sub('Regional Bureau for ','',RBLAC.columns.values[0]).rsplit(' ', 1)[0]
RBLAC.rename(columns = {RBLAC.columns[0]:"Regional_Bureau" }, inplace = True)



In [4]:
#We make a single data frame with all of them together and remove contries with parenteshis. 
RB = RBA.append(RBAP).append(RBAS).append(RBEC).append(RBLAC).reset_index(drop = True)


In [5]:
#Remove parenthesis
RB['Country'] = RB['Country'].str.replace(r"\(.*\)","")
RB['Country'] = RB['Country'].str.rstrip()
RB['Country'] = RB['Country'].str.lstrip()


/var/folders/n5/yyv4zxgx3xb4rdrwt70vgb380000gn/T/ipykernel_22028/730925117.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  RB['Country'] = RB['Country'].str.replace(r"\(.*\)","")


In [6]:
Ed = Ed.drop(columns = ['image_1', 'image_2', 'image_3', 'image_4', 'image_5',
       'image_6', 'image_7', 'image_8', 'image_9'])

In [7]:
Ed.columns = Ed.columns.str.strip()

Ed['Mapper'] = Ed['Mapper'].str.replace('AccLab ', '')
Ed['Mapper'] = Ed['Mapper'].str.replace('Honey Bee Network ', '')
Ed['Mapper'] = Ed['Mapper'].str.replace("[\(\)]",'')

Ed['Mapper'] = Ed['Mapper'].str.replace('Lao','Lao PDR')
Ed['Mapper'] = Ed['Mapper'].str.replace('Pacific','Pacific-Fiji')

for c in Ed.columns:
    if Ed.dtypes[c]==object:
        Ed[c] = Ed[c].str.rstrip()
        Ed[c] = Ed[c].str.lstrip()
    
for tag in range(1,6):
    Ed['What thematic tags apply to this solution? Tag '+str(tag)] = \
    Ed['What thematic tags apply to this solution? Tag '+str(tag)].str.lower()

Ed['What is the purpose of the solution? (brief problem & solution description)']=\
Ed['What is the purpose of the solution? (brief problem & solution description)'].str.lower()
Ed['title']=Ed['title'].str.lower()


/var/folders/n5/yyv4zxgx3xb4rdrwt70vgb380000gn/T/ipykernel_22028/3713225733.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  Ed['Mapper'] = Ed['Mapper'].str.replace("[\(\)]",'')


In [8]:
Ed.Latitude = pd.to_numeric(Ed["Latitude"], downcast="float",errors = 'coerce')
Ed.Longitude = pd.to_numeric(Ed["Longitude"], downcast="float",errors = 'coerce')

In [9]:
ED = pd.merge(Ed, RB, how='left', left_on = 'Mapper', right_on = 'Country')

In [10]:
W = [['solar'],['cooking','stove','oven','ovens'],['wind'],['renewable','reneweable'],['waste'],['recycling'],['biofuel']]
for word in W:
    ED[word[0]] =\
    (ED['What is the purpose of the solution? (brief problem & solution description)'].str.contains('|'.join(word))==True)*1+\
    (ED['title'].str.contains('|'.join(word))==True)*1+\
    (ED['What thematic tags apply to this solution? Tag 1'].str.contains('|'.join(word))==True)*1+\
    (ED['What thematic tags apply to this solution? Tag 2'].str.contains('|'.join(word))== True)*1+\
    (ED['What thematic tags apply to this solution? Tag 3'].str.contains('|'.join(word))== True)*1+\
    (ED['What thematic tags apply to this solution? Tag 4'].str.contains('|'.join(word))== True)*1+\
    (ED['What thematic tags apply to this solution? Tag 5'].str.contains('|'.join(word))== True)*1

    ED[word[0]]= (ED[word[0]]>0)*1

In [11]:
#ED['Energy source'].unique()

In [12]:
ED.loc[ED['Clean cooking application (yes)']=='x','Clean cooking application (yes)'] = 1
ED.loc[ED['Clean cooking application (yes)'].isnull(),'Clean cooking application (yes)'] = 0
ED['Clean cooking application (yes)']= (ED['Clean cooking application (yes)']>0)*1
ED['Clean cooking application (yes)'] = ED['Clean cooking application (yes)'] + ED.cooking
ED['Clean cooking application (yes)'] = (ED['Clean cooking application (yes)'] > 0)*1

In [13]:
aux = ED[ED['Energy source']=='Household application']
ED['Household application']=0
ED.loc[aux.index,'Household application']=1
E_S = [['renewable','Renewable general'],['biofuel','Biological'],['wind','Wind'],['solar','Solar']]
for energy in E_S:
    for i in aux.index:
        if ED.loc[i,energy[0]]>0:
            ED.loc[i,'Energy source']=energy[1]
#The remaining House Hold application convert to null.  
ED.loc[ED['Energy source']=='Household application','Energy source'] =np.nan
ED.loc[ED['Energy source'].isnull(),'Energy source'] = 'Unknown'


In [14]:
#Data frame to create Plot of energy source by country and region. 
Tby_C = ED[['id','Country']].groupby(by = ['Country']).count().reset_index(drop = False)
Tby_C = Tby_C.rename(columns = {'id':'Projects'})
Tby_C = pd.merge(Tby_C, RB, how='left')
Tby_C.head()
TotalR = ED[['id','Region']].groupby(by = "Region").count().reset_index().rename(columns={'id':'Total Region'})
Tby_C = pd.merge(Tby_C,TotalR,how = 'left')
TotalC = ED[['id','Country']].groupby(by = "Country").count().reset_index().rename(columns={'id':'Total Country'})
Tby_C = pd.merge(Tby_C,TotalC,how = 'left')


In [15]:
#Data frame to create Plot of energy source by country and region. 
by_C = ED[['id','Country','Energy source']].groupby(by = ['Country','Energy source']).count().reset_index(drop = False)
by_C = by_C.rename(columns = {'id':'Projects'})
by_C = pd.merge(by_C, RB, how='left')
by_C.head()
TotalR = ED[['id','Region']].groupby(by = "Region").count().reset_index().rename(columns={'id':'Total Region'})
by_C = pd.merge(by_C,TotalR,how = 'left')
TotalC = ED[['id','Country']].groupby(by = "Country").count().reset_index().rename(columns={'id':'Total Country'})
by_C = pd.merge(by_C,TotalC,how = 'left')


In [16]:
#Data frame to get total percentage of Energy source
by_E = by_C.groupby(by = "Energy source").sum().reset_index().drop(columns = ['Total Region','Total Country']) 
by_E['Percentage']= 100*np.round(by_E['Projects']/sum(by_E['Projects']),2)


In [17]:
#Data Frame to get energy source of clean cooking projects. 
CC = ED[ED['Clean cooking application (yes)']>0]
CC_C = CC[['id','Country','Energy source']].groupby(by = ['Country','Energy source']).count().reset_index(drop = False)
CC_C = CC_C.rename(columns = {'id':'Projects'})
CC_C = pd.merge(CC_C, RB, how='left')
CC_C.head()
TotalR = CC[['id','Region']].groupby(by = "Region").count().reset_index().rename(columns={'id':'Total Region'})
CC_C = pd.merge(CC_C,TotalR,how = 'left')
TotalC = CC[['id','Country']].groupby(by = "Country").count().reset_index().rename(columns={'id':'Total Country'})
CC_C = pd.merge(CC_C,TotalC,how = 'left')


In [18]:
#Data frame to get percentages of energy sources of clean cooking projects.
CC_E = CC_C.groupby(by = "Energy source").sum().reset_index().drop(columns = ['Total Region','Total Country']) 
CC_E['Percentage']= 100*np.round(CC_E['Projects']/sum(CC_E['Projects']),2)


In [19]:
#Change name of columns for easier use
ED.rename(columns = {'This solution is protected by Intellectual Property': 'IP',
                    'This solution is Do it Yourself / open source':'DIY',
                    'This solution is a Prototype': 'Prototype',
                    'This solution is a Product': "Product"},inplace = True)

In [20]:
#Find solutions that state if its DIY or Intelectual Property and if its Prototype or Product.
ED.loc[ED['IP']== 'x','IP']=1
ED.loc[ED.IP.isnull(),'IP']=0
ED.loc[ED.DIY== 'x','DIY']=1
ED.loc[ED.DIY.isnull(),'DIY']=0

ED.loc[ED['Prototype']== 'x','Prototype']=1
ED.loc[ED.Prototype.isnull(),'Prototype']=0
ED.loc[ED.Product== 'x','Product']=1
ED.loc[ED.Product.isnull(),'Product']=0


In [21]:
#Create data frame with the ratios of DIY and Prototype. 
A = pd.concat([ED[['DIY','Country']].groupby(by = 'Country').sum(),
           ED[['IP','Country']].groupby(by = 'Country').sum()], axis=1)
A['DIY_IP'] = A.DIY + A.IP

B = pd.concat([ED[['Prototype','Country']].groupby(by = 'Country').sum(),
           ED[['Product','Country']].groupby(by = 'Country').sum()], axis=1)
B['Prot_Prod'] = B.Prototype+B.Product

DF = pd.concat([A,B],axis = 1)
DF = DF[DF.Prot_Prod > 0]
DF['DIY_ratio'] = DF.DIY/DF.DIY_IP
DF['Prototype_ratio']= DF.Prototype/DF.Prot_Prod
DF = pd.merge(DF.reset_index(),RB,how = 'left')


In [22]:
ED[['id','What Sustainable Development Goal is this Solution addressing? Tag 1']]
AUX = pd.DataFrame(columns = {'id'})
AUX.id= ED.id
AUX['Country'] = ED.Country
AUX['Region'] = ED.Region
for n in range(1,18):
    AUX[str(n)] = 0
    ind = ED.query('`What Sustainable Development Goal is this Solution addressing? Tag 1` == '+str(n)+'or\
    `What Sustainable Development Goal is this Solution addressing? Tag 2` == '+str(n)+' or\
    `What Sustainable Development Goal is this Solution addressing? Tag 3` == '+str(n)+' or\
    `What Sustainable Development Goal is this Solution addressing? Tag 4` ==  '+str(n)+' or\
    `What Sustainable Development Goal is this Solution addressing? Tag 5` ==  '+str(n)).index
    AUX.loc[ind,str(n)] = 1


    

In [23]:
AUX2 = AUX.groupby(by = 'Country').sum().reset_index()
AUX2 = AUX2.melt(id_vars=['Country'], value_vars=['1', '2','3','4','5','6','7','8','9'
                                          ,'10','11','12','13','14','15','16','17'])
AUX2.rename(columns={'variable':'Sustainable_Goal','value':'Num_Projects'},inplace = True)
SG = AUX2.merge(RB,how = 'left')

SG["Sustainable_Goal"]= pd.to_numeric(SG["Sustainable_Goal"])
SG = SG.sort_values(by = ['Region','Country'])
AUX2 = SG.groupby(by =['Sustainable_Goal','Region']).sum().reset_index().rename(columns={'Num_Projects':'Projects_Region'})
SG = pd.merge(SG,AUX2,how = 'left',on =['Sustainable_Goal','Region'])



Each energy project is a row and has 35 columns of information (after removing the images columns), such as the country the energy solution is from, what kind of energy source does it use, If it's already a product or a prototype, the cost of the solution, if the solution is DIY, what sustainable goals its addresing, if its a clean cooking aplication and so on. 

## Questions to ask the data. 
Viz4social good already gives the questions to ask the data, they are many and broad so I will focus on a couple of them, as they are good questions and what the organization needs from the visualization.  Such questions are:

- Where are the solutions coming from? What is their distribution per country & per region?

- What type of energy source is more prevalent, what is less? Are there differences per region, and why?

- What are global commonalities across solutions; what are typical applications & use cases for solutions? Are there patterns that emerge when looking at the distribution per country & per region? 

- What overall challenges are the solutions addressing or contributing to overcome?

- Which Sustainable Development Goals are the solutions advancing in particular, and how?

- Looking at the use case of clean cooking solutions, what is their prevalence, distribution, and source of energy?

If applicable / information is available*:

- How are the solutions solving a specific community problem? What are the solutions addressing or contribute to solving? (if applicable)

- How can we display the more quantitative information (ratio of IP vs DIY solutions; Prototype vs Product) in an appealing way that signifies the availability of solutions in country? (if applicable)



The presentation with visualizations ends in October 30 so there are no current visualizations for this data available yet. 

## Tasks
The tasks pusued will be:
1. Identifying the geographic areas the solutions are comming from and the types of energies used geographically. 
 - The goal of the task is learning from which areas are more solutions comming and from which less, maybe to see where would it be best to invest in such solutions and where to help develop new solutions. 
 - The means or how would be comparing the number of solutions per country and regional area.
 - The task seek to learn how many solutions exist per region. 
 - It operates on the data gathered by the mappers that says where each solution comes from. 
 - Workflow would be to be able to zoom in by region to see solutions by country. 
 - The task would be executed by a general user. 
 
2. Identifying if they are general patterns of the type of energies used geographically for Clean Cooking solutions.  
 - To see if some energies are more prevalent in different regions and what countries in the region could need more of it. 
 - By looking at the types of energies in a map.
 - Where are energies more prevalent. 
 - Same data as before plus the data column specifing the kind of energies.
 - The workflow 
 - The task is also executed by the general user. 


3. Looking at the ratios of DYI vs IP and Prototype vs Product by region and country. 
 - To see what kind of projects are more prevalent by region and country. 
 - The means would be looking at the ratios of both things by country in a scatter plot.
 - Worlflow is to see if there is any trend by regions.
 - The task would be executed by general user. 
 
 4. Look at sustainable goals approached by the solutions. 
 - Goal. Observe what sustainable goals are more approached by region. 
 - The means by a scatter plot. 
 - Workflow you can check by region or country. 
 - Task is done by general user. 
 

In [24]:

# Let's implement filtering using dynamic queries. 
selection = alt.selection(type="multi", fields=["Region"])

# Create a container for our two different views
base =  alt.Chart(Tby_C,title = "Energy Solutions per Country").properties(width=500, height=250)

# Let's specify our overview chart
overview = alt.Chart(Tby_C, title="Energy Solutions per Region").mark_bar().encode(
    y = "sum(Projects)",
    x = alt.X("Region:N",sort = '-y'),
    color=alt.condition(selection, alt.value("orange"), alt.value("lightgrey")),
    tooltip=["Region", "sum(Projects)"]
).add_selection(selection).properties(height=250, width=250)

# Create a detail chart
detail = hist = base.mark_bar().encode(
    y = "Projects", 
    x = "Country",
    tooltip = ["Country", "Projects"]
).transform_filter(selection).properties(height=250, width=250)

overview | detail

alt.HConcatChart(...)

In [25]:
# Create a container for our two different views
# Let's implement filtering using dynamic queries. 
selection = alt.selection(type='multi', fields=['Region'], on='mouseover', nearest=True)
base =  alt.Chart(by_C, title = "Solutions Energy Sources by Country").properties(width=500, height=500)

general = alt.Chart(by_E, title = 'Percentage of Solutions Energy sources').mark_arc().encode(
    theta=alt.Theta(field="Projects", type="quantitative"),
    color=alt.Color(field="Energy source", type="nominal"),
    tooltip = ['Energy source', 'Percentage']
).properties(height=150, width=600)


overview = alt.Chart(by_C, title = "Solutions Energy Sources by Region").mark_bar().encode(
    x='Region',
    y='sum(Projects)',
    color='Energy source',
    opacity = alt.condition(selection, alt.value(1.0), alt.value(0.3)),
    order=alt.Order(
      # Sort the segments of the bars by this field
      'sum(Projects)',
      sort='descending'
    ),
     #tooltip=["Region","Total Region" ,"sum(Projects)","Energy source"]
    tooltip = ["Region","Total Region" ,"Energy source",
               alt.Tooltip('sum(Projects)', title='Projects by Energy Source')]
).add_selection(selection).properties(height=250, width=200)


# Create a detail chart
detail = hist = base.mark_bar().encode(
    y = "Projects", 
    x = "Country",
    #color = 'Energy source',
    color = alt.Color('Energy source', scale=alt.Scale(scheme='set1')),
    order=alt.Order('Projects',sort='descending'),
    tooltip = ["Country", "Total Country","Energy source",
               alt.Tooltip('Projects', title='Projects by Energy Source')]
).transform_filter(selection).properties(height=250, width=350)

general & (overview | detail)


alt.VConcatChart(...)

In [26]:
from vega_datasets import data
selection = alt.selection(type='multi', fields=['Region'], on='mouseover', nearest=True)
countries = alt.topo_feature(data.world_110m.url, 'countries')

background = alt.Chart(countries,title = "Where are the solutions geographically Located").mark_geoshape(
    fill='white',
    stroke='lightgray'
).project(
    "equirectangular"
).properties(
    width=600,
    height=400
)


points = alt.Chart(ED).mark_circle().encode(
    longitude='Longitude:Q',
    latitude='Latitude:Q',
    size=alt.value(20),
    color = alt.Color('Energy source', scale=alt.Scale(scheme='set1')),
    tooltip=["Region","Country"],
    opacity=alt.condition(selection,alt.value(.8),alt.value(.1))
).add_selection(selection)


background + points


alt.LayerChart(...)

In [27]:
# Implementing selection
selection = alt.selection(type='multi', fields=['Region'])

alt.Chart(DF, title = 'Are the Solutions DIY or have Intelectual rights,\
are they a Prototype or already a Product').mark_circle().encode(
    x = "Prototype_ratio",
    y    = 'DIY_ratio',
    color=alt.Color('Region', scale=alt.Scale(scheme='accent')),
    size=alt.value(100),
    tooltip=["Country", "DIY",'IP','Prototype','Product'],
    opacity=alt.condition(selection,alt.value(1),alt.value(.2))
).add_selection(selection)

alt.Chart(...)

#### What sustainable goal are the solutions addresing the most by country and region?

In [28]:
selection = alt.selection(type='multi', fields=['Region'])

overview = alt.Chart(SG).mark_circle().encode(
    x=alt.X('Region', sort=['Region']),
    y    = alt.Y('Sustainable_Goal'),
    color=alt.Color('Region', scale=alt.Scale(scheme='set1')),
    size='Projects_Region',
    tooltip=["Region",'Sustainable_Goal','Projects_Region'],
    opacity=alt.condition(selection,alt.value(1),alt.value(.2))
).add_selection(selection)

detail = alt.Chart(SG, title = 'What sustainable goasl are the solutions adressing the Most').mark_circle().encode(
     x=alt.X('Country', sort=['Region']),
    y    = 'Sustainable_Goal',
    color=alt.Color('Region', scale=alt.Scale(scheme='set1')),
    size='Num_Projects',
    tooltip=['Country','Sustainable_Goal','Num_Projects'],
    opacity=alt.condition(selection,alt.value(1),alt.value(.2))
).transform_filter(selection)
    
overview | detail


alt.HConcatChart(...)

In [29]:
df = ED.loc[ED.id.isin(AUX[AUX['7']>0].id),['What thematic tags apply to this solution? Tag 1',
       'What thematic tags apply to this solution? Tag 2',
       'What thematic tags apply to this solution? Tag 3',
       'What thematic tags apply to this solution? Tag 4',
       'What thematic tags apply to this solution? Tag 5']]

df = df.stack().reset_index().rename(columns = {0:'Tags'}).drop(columns = ['level_0','level_1'])
df = df[~df.Tags.isnull()].reset_index(drop = True)

T = pd.DataFrame(columns = ['word','Counter'])

T.word = df.Tags.unique()
T.Counter = 0
for i in range(0,len(df)):
    ind = T[T.word == df.Tags[i]].index
    T.loc[ind.item(),'Counter'] =T.loc[ind.item(),'Counter']+1



#### What are the key tags of the solutions for Sustainable Goal 7?
Sustainable Goal 7 is affordable and Clean Energy. 


In [30]:
#Keep solutions with only 7 as sustainable goal.



import random
def shuffled_range(n): return random.sample(range(n), k=n)
n = len(T)  # words_and_counts: a pandas data frame
x = shuffled_range(n)
y = shuffled_range(n)

data = T.assign(x=x, y=y)

base = alt.Chart(data).encode(
    x=alt.X('x:O', axis=None),
    y=alt.Y('y:O', axis=None)
).configure_view(strokeWidth=0)  # remove border

word_cloud = base.mark_text(baseline='middle').encode(
    text='word:N',
    color=alt.Color('Counter:Q', scale=alt.Scale(scheme='goldred')),
    size=alt.Size('Counter:Q', legend=None)
).properties(
    width=400,
    height=400
)

word_cloud

alt.Chart(...)

In [31]:
p_cook = ED['Clean cooking application (yes)'].sum()/len(ED['Clean cooking application (yes)'])


#### A third of the solutions are for Clean Cooking. What energy sources do the solutions use for such applications?

In [32]:
# Create a container for our two different views
# Let's implement filtering using dynamic queries. 
selection = alt.selection(type='multi', fields=['Region'])
base =  alt.Chart(CC_C, title = 'By Country').properties(width=500, height=500)

general = alt.Chart(CC_E,title = 'Energy sources of Clean Cooking Solutions').mark_arc().encode(
    theta=alt.Theta(field="Projects", type="quantitative"),
    color=alt.Color(field="Energy source", type="nominal"),
    tooltip = ['Energy source', 'Percentage']
).properties(height=150, width=600)


overview = alt.Chart(CC_C, title = "By region").mark_bar().encode(
    x='Region',
    y='sum(Projects)',
    color='Energy source',
    opacity = alt.condition(selection, alt.value(1.0), alt.value(0.3)),
    order=alt.Order(
      # Sort the segments of the bars by this field
      'sum(Projects)',
      sort='descending'
    ),
     #tooltip=["Region","Total Region" ,"sum(Projects)","Energy source"]
    tooltip = ["Region","Total Region" ,"Energy source",
               alt.Tooltip('sum(Projects)', title='Projects by Energy Source')]
).add_selection(selection).properties(height=250, width=200)


# Create a detail chart
detail = hist = base.mark_bar().encode(
    y = "Projects", 
    x = "Country",
    #color = 'Energy source',
    color = alt.Color('Energy source', scale=alt.Scale(scheme='accent')),
    order=alt.Order('Projects',sort='descending'),
    tooltip = ["Country", "Total Country","Energy source",
               alt.Tooltip('Projects', title='Projects by Energy Source')]
).transform_filter(selection).properties(height=250, width=350)

general & (overview | detail)


alt.VConcatChart(...)

### Key Elements of the Design
For the design I tried to focus in easy to understand plots. 
Used bar plots to compare the number of solutions with an interactive feature so you could see how the regions solutions decompose by country.  At the start all the regions on the left are in orange and to the rights there is a bar for each country. If you click on the left plot on a region bar the right plot changes to only show the countries in such region. For each bar if you put the cursosr above it gives you the info of what country it is and how many solutions they registered. 

To see what kind of energies where more used, I added stacked bars to the previousvisualization so the total length of the bar would still be the total number of solutions and the colors represent the fraction of the solutions for a given energy type. In this case the tooltip gives you both the total number of solutions and how many solutions of this specific contry.  At the top of the two stacked bars I added a circle plot showing the total percentage of each solution and the toolbar tells you the kind of energy and percentage of such energy. 

To complete this viz I added a map of the world with the points based on the latitued and longitude the solutions are registered, this in order to get a better approach of the geography of where the solutions are located. 

I repeated the solution of the stacked bars to the data of the clean cooking solutions to be able to compare it with the general data.  

For DIY and prototypes I choose a scatter plot as the medium and one point per country with the color representing the region and being able to select by region to see if there is any trend by region. The y axis represent the fraction of DIY/(DIY+Intelectual Property) while the y axis is the ratio of Prototypes/(Prototypes+Product). I choose this design to sse if there was any trend between thoose two aspects of the projects or any regional trend.   

Finally to see which sustainable goals where more approached I did two scatter plots one by region that if you click would see the one by country. I use color to group by region and size of the mark to show how many solutions had the goal. From the plot it was clear number 7 was the most approached so I lloked at a word plot to see what kind of tags where more often added to solutions addresing sustainable goal number 7. 




### Evaluation Approach
For the evaluation approach as there wont be a lot of people evaluating I though the think aloud would be the best approach and to see what they think of the different ways the data is visualized and what insights they get from it. 
Three persons interacted with the visualizations, and I asked them to say everything they where thinking and discovering while they interacted with the visualizations. 

There were 2 females and a male, two above 70 and the other one in her 40s. 
For the elder ones tha interaction wasn't intitive at all, they didn't click on the visualizations even when told and had a bit of a hard time with the colors. They still got the main insights of the graohs showing the numbers of solutions by country and region, all noticed that most of the solutions where in Africa and that India was the country with more solutions. They also noticed that there are large regions of the world with out any solutions reported and specifically that the solutions came only from Developing contries.  When showed by type of energy one found the colors confusing as he couldnt tell between orange and pink, and that when all the countries were selected we couldnt tell between the countries. Other one appreciate tha solar was yellow, unknow was grey and hydro blue, but that didnt repeated in the clean cooking ones. 

They all found the scatter ratios plot confusing and didnt see the point of it. 

For the 17 UN Sutainable Goals plot all needed an explanation of what the y axis was as neither of them knew anything about the UN sustainable goals, so if the visualization is for the general public I would need to add a chart showing what this goals are and its number. One suggested that the color didnt help her understand the data and would be better if the color represnts the goal instead of the region of the country. 

For the clean cooking the main message for the 3 of them was that there wasn't enough infromation about the energy sources as half of them dont have the energy source stated in the original data. 

Overall the evaluation gave me a lot of points of how I could improve the visualizations. 

For the selection, the older ones prefered the mouseover to get more info as the clicking wasn't intuitive fro them at all, however for the younger one the mouse over was annoying as when moving the coursor to focus on a detail the plot on the left would change without her wanting to do that. 


### Conclusions

What worked best of the visualizations where the bar plots showing the number of solutions and the focus of the country specially with the toolbar showing the data of each bar or piece of bar. The map gaves more of a geograpicall sense of where there is no data, but because the latitude and longitude data wasn't perfect in the data set it also made it a bit confusing. 

Both scatter plots would need more context and explanation to be usefull for the general public. Probably the visualization should include the UN infographic about their sustainable goals, what each number is.   

Given that the clean cooking data had too many unknows as energy sources showing the stacked bars wasnt as effective as the main insigith you get when seeing it is that more than half od the energy sources are unknow so the details of which regions and countries have more clean cooking solutions independently of the energy source got lost. To improve it I would change it to a simple bar plot only showing the number of clean cooking solutions or one showing the toal and with a color showing which of such solutions are clean cooking by country. So instead of having to check betwen the two visualizations, one for Total and one for cleaning, one single would have all the data. 